### Suncal-Web Distribution Explorer

---

In [1]:
%matplotlib ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import sympy

import suncalwidgets as sw
import suncal
from suncal import dist_explore
suncal.plotting.plotstyle.update({'font.size': 9})

In [2]:
class Dist(widgets.VBox):
    def __init__(self):
        self.name = widgets.Text('a', description='Name', continuous_update=False)
        self.dist = sw.DistributionWidget()
        self.btnsample = widgets.Button(description='Sample')
        self.name.observe(self.checkname)
        super().__init__([self.name, self.dist, self.btnsample])

    def checkname(self, change):
        try:
            expr = sympy.sympify(self.name.value)
            n = len(expr.free_symbols)
        except ValueError:
            self.dist.layout.visibility = 'hidden'
            self.btnsample.disabled = True
        else:
            self.btnsample.disabled = False
            if hasattr(expr, 'is_symbol') and expr.is_symbol:
                # Single variable
                self.dist.layout.visibility = 'visible'
                self.btnsample.description = 'Sample'
            else:  # Monte Carlo
                self.dist.layout.visibility = 'hidden'
                self.btnsample.description = 'Calculate'


class DistList(widgets.VBox):
    def __init__(self):
        self.displayedname = None
        self.btnadd = widgets.Button(description='+', layout={'width': 'max-content'})
        self.btnrem = widgets.Button(description='–', layout={'width': 'max-content'})
        self.btnadd.on_click(self.additem)
        self.btnrem.on_click(self.remitem)
        self.btnlayout = widgets.HBox([self.btnadd, self.btnrem])
        self.dists = widgets.Accordion()
        self.nsamples = widgets.IntText(value=10000, description='Samples')
        self.nsamples.observe(self.on_samplechange)
        self.cmbFit = widgets.Dropdown(options=['None', 'normal', 't', 'triangular', 'gamma', 'uniform', 'expon'], description='Fit')
        self.chkProbPlot = widgets.Checkbox(description='Show Probability Plot')
        self.chkCov = widgets.Checkbox(description='Show 95% Coverage')
        self.chkProbPlot.observe(self.replot)
        self.chkCov.observe(self.replot)
        self.cmbFit.observe(self.replot)
        self.additem()
        super().__init__([self.btnlayout, self.dists, self.nsamples, self.cmbFit, self.chkProbPlot, self.chkCov])

    def on_samplechange(self, change):
        dexp.set_numsamples(self.nsamples.value)

    def additem(self, change=None):
        n = len(self.dists.children)
        newdist = Dist()
        self.dists.children = list(self.dists.children) + [newdist]
        self.dists.set_title(n, f'Distribution {n+1}')
        self.dists.selected_index = n
        newdist.btnsample.on_click(lambda change: self.sample(newdist.name.value, newdist.dist.get_dist(), change))

    def remitem(self, change=None):
        self.dists.children = list(self.dists.children)[:-1]
    
    def get_dists(self):
        dists = {}
        for child in self.dists.children:
            dists[child.name.value] = child.dist.get_dist()
        return dists

    def sample(self, name, dist, change):
        self.displayedname = name
        dexp.dists[name] = dist
        dexp.sample(name)
        with outtable:
            outtable.clear_output()
            display(dexp.out.report_single(name))
        dexp.out.plot_hist(name, plot=fig, qqplot=self.chkProbPlot.value, coverage=self.chkCov.value)

    def replot(self, change):
        if self.displayedname:
            with outtable:
                outtable.clear_output()
                display(dexp.out.report_single(self.displayedname))
            fitdist = None if self.cmbFit.value == 'None' else self.cmbFit.value
            if fitdist is None and self.chkProbPlot.value:
                fitdist = 'normal'
            dexp.out.plot_hist(self.displayedname, plot=fig, fitdist=fitdist,
                               qqplot=self.chkProbPlot.value, coverage=self.chkCov.value)
        
        

In [3]:
dexp = dist_explore.DistExplore()

plt.ioff()
fig = plt.figure()
plt.ion()
outtable = widgets.Output()

dlist = DistList()
outlayout = widgets.VBox([fig.canvas, outtable])
widgets.HBox([dlist, outlayout])

---

Use this tool to perform manual Monte Carlo analysis on different probability distributions. Mainly used for training exercises.

---

#### Suncal

Suncal was developed by the Primary Standards Lab at Sandia National Laboratories. For more features and desktop version, see [https://sandiapsl.github.io](https://sandiapsl.github.io).

© 2019-2021 National Technology & Engineering Solutions of Sandia, LLC (NTESS).
Under the terms of Contract DE-NA0003525 with NTESS, the U.S. Government
retains certain rights in this software.